In [ ]:
import pandas as pd
import numpy as np
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import LSTM, Dense, Dropout
from tensorflow.keras.utils import to_categorical
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler, LabelEncoder
import joblib # Import joblib for saving/loading scaler and label_encoder
import os # For creating directories and managing files

# --- Configuration ---
SENSOR_HZ = 67
TIME_STEPS = int(SENSOR_HZ * 0.75)
if TIME_STEPS == 0:
    TIME_STEPS = 1
print(f"Calculated TIME_STEPS: {TIME_STEPS}")

N_FEATURES = 6 # accel_x, accel_y, accel_z, gyro_x, gyro_y, gyro_z

# --- Define paths for data and models ---
# It's good practice to define these relative to the script
DATA_DIR = 'data'
MODELS_DIR = 'models'
CSV_FILE_NAME = 'gesture_datas1.csv'
CSV_FILE_PATH = os.path.join(DATA_DIR, CSV_FILE_NAME)

MODEL_H5_PATH = os.path.join(MODELS_DIR, 'simple_gesture_recognition_model.h5')
LABEL_ENCODER_PATH = os.path.join(MODELS_DIR, 'simple_label_encoder.pkl')
SCALER_PATH = os.path.join(MODELS_DIR, 'simple_scaler.pkl')

# Ensure directories exist
os.makedirs(DATA_DIR, exist_ok=True)
os.makedirs(MODELS_DIR, exist_ok=True)


# --- 1. Data Preprocessing (Offline - for your CSV) ---

def load_and_preprocess_data(filepath, time_steps, n_features):
    print(f"Loading data from: {filepath}")
    if not os.path.exists(filepath):
        print(f"Error: Data file not found at {filepath}")
        print("Please ensure 'gesture_datas1.csv' is in the 'data/' directory.")
        raise FileNotFoundError(f"CSV file not found: {filepath}")

    df = pd.read_csv(filepath)

    sensor_features = ['accel_x', 'accel_y', 'accel_z', 'gyro_x', 'gyro_y', 'gyro_z']
    
    if not all(feature in df.columns for feature in sensor_features):
        print(f"Error: Missing one or more expected sensor columns. Expected: {sensor_features}")
        raise ValueError("Missing sensor data columns in CSV.")
    if 'gesture_label' not in df.columns:
        print("Error: Missing 'gesture_label' column in CSV.")
        raise ValueError("Missing 'gesture_label' column in CSV.")

    X = df[sensor_features].values
    y = df['gesture_label'].values # Assuming you have a 'gesture_label' column

    label_encoder = LabelEncoder()
    y_encoded = label_encoder.fit_transform(y)
    num_classes = len(label_encoder.classes_)
    print(f"Detected classes: {label_encoder.classes_}")
    print(f"Number of classes: {num_classes}")
    
    scaler = StandardScaler()
    X_scaled = scaler.fit_transform(X)

    sequences = []
    labels = []
    # Using a stride of 1 to create overlapping sequences for more training data
    for i in range(0, len(X_scaled) - time_steps + 1, 1):
        sequence = X_scaled[i:i + time_steps]
        # Label the sequence based on the last data point's label
        labels.append(y_encoded[i + time_steps - 1]) 
        sequences.append(sequence)

    X_seq = np.array(sequences)
    y_seq = np.array(labels)

    y_one_hot = to_categorical(y_seq, num_classes=num_classes)

    print(f"Shape of X_seq (sequences): {X_seq.shape}")
    print(f"Shape of y_one_hot (one-hot labels): {y_one_hot.shape}")
    return X_seq, y_one_hot, label_encoder, scaler, num_classes

# --- 2. Build a Simpler Neural Network Model ---

def build_simple_model(time_steps, n_features, num_classes):
    model = Sequential([
        # LSTM layer processes sequences. return_sequences=False means output is a single vector per sequence.
        LSTM(units=64, input_shape=(time_steps, n_features), return_sequences=False),
        Dropout(0.3),
        # Dense layers for classification
        Dense(units=32, activation='relu'),
        Dropout(0.2),
        # Output layer with softmax for multi-class classification
        Dense(units=num_classes, activation='softmax')
    ])

    model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])
    return model

# --- 3. Training the Model ---

def train_model(model, X_train, y_train, X_val, y_val, epochs=50, batch_size=128): # Increased epochs slightly
    print("Training model...")
    # Early stopping to prevent overfitting and save best weights
    early_stopping = tf.keras.callbacks.EarlyStopping(monitor='val_loss', patience=15, restore_best_weights=True) # Increased patience
    
    history = model.fit(X_train, y_train, 
                        epochs=epochs, 
                        batch_size=batch_size, 
                        validation_data=(X_val, y_val),
                        callbacks=[early_stopping],
                        verbose=1) # Set verbose to 1 to see progress during training
    print("Training complete.")
    return history

# --- Function to generate requirements.txt ---
def generate_requirements_txt():
    # This will capture the exact versions of installed packages
    # It's better to run 'pip freeze > requirements.txt' in your environment
    # but for a quick script-based generation, we can list common ones.
    
    # You should manually verify this against your 'pip freeze' output
    # to ensure exact versions are captured for reproducibility.
    
    requirements_content = """
pandas
numpy
tensorflow
scikit-learn
joblib
"""
    with open('requirements.txt', 'w') as f:
        f.write(requirements_content.strip())
    print("Generated requirements.txt")

# --- Main execution for training ---
if __name__ == '__main__':
    # Generate requirements.txt first
    generate_requirements_txt()

    try:
        X_seq, y_one_hot, label_encoder, scaler, NUM_CLASSES = load_and_preprocess_data(CSV_FILE_PATH, TIME_STEPS, N_FEATURES)
    except (ValueError, FileNotFoundError) as e:
        print(f"Data loading failed: {e}")
        print("Please ensure your CSV file is correctly formatted and present in the 'data/' directory.")
        exit()

    # Split data into training and validation sets
    X_train, X_val, y_train, y_val = train_test_split(X_seq, y_one_hot, test_size=0.2, random_state=256, stratify=y_one_hot)

    # Build the simpler model
    model = build_simple_model(TIME_STEPS, N_FEATURES, NUM_CLASSES)
    model.summary()

    # Train the model
    history = train_model(model, X_train, y_train, X_val, y_val)

    # Evaluate the model on the validation set
    loss, accuracy = model.evaluate(X_val, y_val, verbose=0)
    print(f"\n--- Model Evaluation ---")
    print(f"Validation Loss: {loss:.4f}")
    print(f"Validation Accuracy: {accuracy:.4f}")

    # Save the trained model, label encoder, and scaler for live prediction
    model.save(MODEL_H5_PATH)
    joblib.dump(label_encoder, LABEL_ENCODER_PATH)
    joblib.dump(scaler, SCALER_PATH)
    print(f"\nModel saved to: {MODEL_H5_PATH}")
    print(f"Label Encoder saved to: {LABEL_ENCODER_PATH}")
    print(f"Scaler saved to: {SCALER_PATH}")